# Redes neuronales

### MultiLayer Perceptron
- - -

Puntos clave del Perceptrón Multicapa:

- Capacidad para aprender modelos no lineales.

- Diferentes inicializaciones de peso aleatorio pueden llevar a una precisión de validación diferente.

- MLP requiere ajustar una serie de hiperparámetros (p.ej.,  el número de neuronas ocultas, capas e iteraciones).

- MLP es sensible a la escala de las características.

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

El dataset Sonar conta de datos obtenidos del rebote se señales en objetos enterrados, este dataset implica la predicción de si un objeto es una mina (cilindro metálico) o una roca, dada la fuerza de los retornos del sonar en diferentes ángulos.

In [ ]:
sonar = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/sonar.csv")

In [ ]:
sonar.head()

In [ ]:
print("SHAPE DATASET\n Número de Filas:\t{}\n Número de Columnas:\t{}".format(sonar.shape[0], sonar.shape[1]))

In [ ]:
X = sonar.drop("Class", 1)
y = sonar["Class"]

In [ ]:
#X = MinMaxScaler().fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [ ]:
MLPC = MLPClassifier(hidden_layer_sizes=(5,5,5), activation='relu', learning_rate_init=0.01, max_iter=1000, batch_size = 200, solver='adam', 
                     random_state=1)

In [ ]:
print("Accuracy en cross-validation:", round(cross_val_score(MLPC, X_train, y_train, cv=5).mean(),2))

In [ ]:
#Vamos a hacer un grid search de distintos parámetros
MLPC = MLPClassifier(early_stopping = True,max_iter=1000,activation='relu', batch_size = 200, solver='adam',random_state=1)
parameters = {
    'hidden_layer_sizes': [(5,5,5),(20,20),(100,100,100)],
    'learning_rate_init' : [0.01, 0.00001]
    }
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(MLPC, parameters, cv=5, verbose = True)
grid_search.fit(X_train, y_train) 
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("He obtenido una accuracy de %0.3f (+/-%0.03f) con los parámetros %r" % (mean, std * 2, params))

In [ ]:
print("los mejores parámetros han sido: ", grid_search.best_params_)

In [ ]:
#Ahora hacemos el fit con todo el dataset de train y vemos qué nos ha dado para el de test.
new_MPLC = MLPClassifier(**grid_search.best_params_,max_iter=1000,activation='relu', 
                                batch_size = 200, solver='adam',random_state=1)
new_MPLC.fit(X_train, y_train)
predicciones = new_MPLC.predict(X_test)
print("Accuracy en test: ", accuracy_score(predicciones, y_test) )

In [ ]:
import matplotlib.pyplot as plt
print(len(new_MPLC.loss_curve_))
plt.plot(new_MPLC.loss_curve_, color = 'Blue')


In [ ]:
#Vemos que la loss seguía decreciendo, vamos a intentar dejarlo entrenando unas iteraciones más.
#añadimos estos parámetros: tol = 0, n_iter_no_change=1000
#Ahora hacemos el fit con todo el dataset de train y vemos qué nos ha dado para el de test.
new_MPLC = MLPClassifier(**grid_search.best_params_, early_stopping = True,
                         max_iter=1000,activation='relu', batch_size = 200, 
                         solver='adam',random_state=1, tol = 0, n_iter_no_change=1000)
new_MPLC.fit(X_train, y_train)
predicciones = new_MPLC.predict(X_test)
print("Accuracy en test: ", accuracy_score(predicciones, y_test) )

In [ ]:
print(len(new_MPLC.loss_curve_))
plt.plot(new_MPLC.loss_curve_, color = 'Blue')


- - -
- - -

Ahora resolveremos un problema de regresión.

El siguiente dataset cuenta con 768 formas de edificios que fueron generadas a partir de realizar una simulación de varios escenarios con diferencias sobre la superficie acristalada, la distribución de la superficie acristalada y la orientación, entre otros parámetros. El objetivo es hacer regresión sobre la eficiencia energética del edificio (Heating Load y Cooling Load).

In [ ]:
energy = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/energy.csv")

In [ ]:
energy.head()

In [ ]:
print("SHAPE DATASET\n Número de Filas:\t{}\n Número de Columnas:\t{}".format(energy.shape[0], energy.shape[1]))

In [ ]:
X = energy.drop("Heating Load", 1)
y = energy["Heating Load"]

In [ ]:
X = MinMaxScaler().fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
MLPR = MLPRegressor(hidden_layer_sizes=(5,5), activation='relu', learning_rate_init=0.001, max_iter=2000, solver='sgd', 
                     random_state=1)

In [ ]:
#Calculamos R2 promedio dividiendo en 5 folds. Cuanto más próxima a 1, mejor.
print("MULTILAYER PERCEPTRON REGRESSOR:", round(cross_val_score(MLPR, X_train, y_train, cv=5).mean(),2))

In [ ]:
#Hacemos lo mismo eliminando también el Cooling load
X = energy.drop(["Heating Load", "Cooling Load"], 1)
y = energy["Heating Load"]
print(X.head(3))


In [ ]:
X = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
MLPR = MLPRegressor(hidden_layer_sizes=(5,5), activation='relu', learning_rate_init=0.001, max_iter=2000, solver='sgd', 
                     random_state=1)
#Calculamos R2 promedio dividiendo en 5 folds. Cuanto más próxima a 1, mejor.
print("MULTILAYER PERCEPTRON REGRESSOR:", round(cross_val_score(MLPR, X_train, y_train, cv=5).mean(),2))